### The Sensing Entities
The entities of the SensorThings API's Sensing component are illustrated in the following figure.

<img src="../assets/images/STA_schema.png" >

To maintain proper references between elements, you must create them in a specific order:

1. Thing
2. Location
3. ObservedProperty
4. Sensor
5. Datastream
6. Observation (+FeatureOfInterest)

### Preliminary Steps

This section contains the preliminary steps to set up the base URL, headers, and import necessary libraries.

In [ ]:
import requests
import json
import re
import istsos4_utils as st
from IPython.display import display, Markdown

# base url
base_url = "http://api:5000/istsos4/v1.1/"

# Headers (optional, but specifying Content-Type ensures proper handling of JSON data)
headers = {
    'Content-Type': 'application/json'
}

meteo = st.sta(base_url)

### Create a Thing

In [ ]:
body = {
    "name" : "FIU_VAL",
    "description" : "Water level, water temperature and water electrical conductivity recorder Ticino river",
    "properties" : {
        "keywords" : "water,river,height,temperature,conductivity,ACSOT",
        "description": "River level, water temperature and water electrical conductivity fiume Ticino valle"
    }
}

# POST request with the JSON body
response = requests.post(base_url + 'Things', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Thing created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Thing @iot.id using a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    thing_id = int(match.group(1))
else:
    print("No number found in parentheses.")

### Create a Location
To create a Location and link it to an existing Thing, you include the Thing's ID in the JSON payload of the Location you are creating.

In [ ]:
body = {
    "name": "fiume Ticino valle",
    "description": "",
    "encodingType": "application/vnd.geo+json",
    "location": {
        "type": "Point",
        "coordinates": [
            8.956099,
            46.172245
        ]
    },
    "Things": [
        { "@iot.id": thing_id}
    ]
}

# POST request with the JSON body
response = requests.post(base_url + 'Locations', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Location created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Location @iot.id using a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    location_id = int(match.group(1))
else:
    print("No number found in parentheses.")

In [ ]:
response = requests.get(f"{base_url}Things({thing_id})?$expand=Locations")
json_data = json.dumps(response.json(), indent=2)
md = f"```json\n{json_data}\n```"
display(Markdown(md))

In [ ]:
things = meteo.query_api('Things','$expand=Locations')
map = meteo.map_things(things)
map

### Create an Observed Property

In [ ]:
body = {
  "name": "ground:water:voltage",
  "description": "Ground water voltage",
  "properties": {},
  "definition": "{}"
}

# POST request with the JSON body
response = requests.post(base_url + 'ObservedProperties', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"ObservedProperty created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Observed Property @iot.id usimng a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    observed_property_id = int(match.group(1))
else:
    print("No number found in parentheses.")

### Create a Sensor

In [ ]:
body = {
  "name": "Ecolog 1000",
  "description": "",
  "properties": {},
  "encodingType": "application/json",
  "metadata": '{"brand": "OTT", "type": "Pressure, temperature, electrical conductivity sensor"}'
}

# POST request with the JSON body
response = requests.post(base_url + 'Sensors', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Sensor created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Sensor @iot.id usimng a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    sensor_id = int(match.group(1))
else:
    print("No number found in parentheses.")

### Create a Datastream
The Datastream requires a Thing, Sensor and ObservedProperty. 

In [ ]:
body = {
  "name" : "V_FIU_VAL",
  "description" : "",
  "observationType": "",
  "unitOfMeasurement": {
    "name": "Voltage",
    "symbol": "V",
    "definition": ""
  },
  "Thing": {"@iot.id": thing_id},
  "Sensor": {"@iot.id": sensor_id},
  "ObservedProperty": {"@iot.id": observed_property_id}
}

# POST request with the JSON body
response = requests.post(base_url + 'Datastreams', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Datastream created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Datastream @iot.id usimng a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    datastream_id = int(match.group(1))
else:
    print("No number found in parentheses.")

In [ ]:
response = requests.get(f"{base_url}Things({thing_id})?$expand=Locations,Datastreams($expand=Sensors,ObservedProperties)")
json_data = json.dumps(response.json(), indent=2)
md = f"```json\n{json_data}\n```"
display(Markdown(md))

### Create an Observation
When creating Observations, the following additional rules apply:

1. If the phenomenonTime is not specified in the JSON payload, the server will automatically assign the current time as the phenomenonTime.
2. If the featureOfInterest is not provided, the server will generate a FeatureOfInterest based on the Location associated with the Thing from the relevant Datastream.

#### Create Observation (Datastream in the JSON and FeatureOfInterest set to the Thing's Location)

In [ ]:
body = {
    "result": 3.63,
    "Datastream": {"@iot.id": datastream_id}
}

# POST request with the JSON body
response = requests.post(base_url + 'Observations', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Observation created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Datastream @iot.id usimng a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    observation_id = int(match.group(1))
else:
    print("No number found in parentheses.")

#### Create Observation (Datastream in the JSON and new FeatureOfInterest in the JSON)

In [ ]:
body = {
    "result" : 3.62,
    "Datastream": {"@iot.id": datastream_id},
    "FeatureOfInterest": {
        "name": "A weather station.",
        "description": "A weather station.",
        "feature": {
            "type": "Point",
            "coordinates": [
                8.956099,
                46.172335
            ]
        },
        "encodingType": "application/vnd.geo+json"
    },
}

# POST request with the JSON body
response = requests.post(base_url + f'Observations', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Observation created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Datastream @iot.id usimng a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    observation_id = int(match.group(1))
else:
    print("No number found in parentheses.")

#### Create Observation (Datastream in the URL and new FeatureOfInterest in the JSON)

In [ ]:
body = {
    "result" : 23,
    "FeatureOfInterest": {
        "name": "A weather station.",
        "description": "A weather station.",
        "feature": {
            "type": "Point",
            "coordinates": [
                8.956229,
                46.172245
            ]
        },
        "encodingType": "application/vnd.geo+json"
    },
}

# POST request with the JSON body
response = requests.post(base_url + f'Datastreams({datastream_id})/Observations', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Observation created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Datastream @iot.id usimng a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    observation_id = int(match.group(1))
else:
    print("No number found in parentheses.")

In [ ]:
response = requests.get(f"{base_url}Things({thing_id})?$expand=Locations,Datastreams($expand=Sensors,ObservedProperties,Observations($top=3;$expand=FeaturesOfInterest))")
json_data = json.dumps(response.json(), indent=2)
md = f"```json\n{json_data}\n```"
display(Markdown(md))

In [ ]:
datastreams = meteo.query_api('Datastreams')
map = meteo.map_datastreams(datastreams)
map

### Creating multiple related entities in one POST
It is possible to create an entities, and its relations, in one POST, by giving the full related entity in the JSON instead of only the entitiy id.

In [ ]:
body = {
    "unitOfMeasurement": {
        "name": "",
        "symbol": "RSSI",
        "definition": ""
    },
    "description": "",
    "name": "RSSI_FIU_VAL",
    "observationType": "",
    "ObservedProperty": {
        "name": "ground:water:signal_strength",
        "definition": "{}",
        "description": "Ground water signal_strength"
    },
    "Sensor": {
        "@iot.id": sensor_id
    },
    "Thing": {
        "@iot.id": thing_id
    },
    "Observations": [
        { 
            "result": 1,
        },
        {
            "result": 1,
            "FeatureOfInterest": {
                "name": "A weather station.",
                "description": "A weather station.",
                "feature": {
                    "type": "Point",
                    "coordinates": [
                        8.956099,
                        46.172135
                    ]
                },
                "encodingType": "application/vnd.geo+json"
            },
        },
        {
            "result": 0,
            "FeatureOfInterest": {
                "name": "A weather station.",
                "description": "A weather station.",
                "feature": {
                    "type": "Point",
                    "coordinates": [
                        8.956219,
                        46.172245
                    ]
                },
                "encodingType": "application/vnd.geo+json"
            },
        }
    ]
}

# POST request with the JSON body
response = requests.post(base_url + 'Datastreams', data=json.dumps(body), headers=headers)

# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print(f"Datastream created successfully ({response.headers['location']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

# Let's get the Thing @iot.id using a regex to extract the number in parentheses
match = re.search(r'\((\d+)\)', response.headers['location'])
if match:
    datastream_id = int(match.group(1))
else:
    print("No number found in parentheses.")

In [ ]:
datastreams = meteo.query_api('Datastreams')
map = meteo.map_datastreams(datastreams)
map